# Set up new Chromium User Profile

This will open Chromium and give you enough time to set up a new user profile. You then find this profile and use it later on in other scripts where your automations are surfing as you. For example, on the Mac, my profile folder got named `Person 1` so there is a line in automation scripts where I have to use the following line. This scripts helps you create that location and find out what it is.

```python
user_data_folder_name = "Person 1"
```

In [ ]:
import asyncio
import getpass
import logging
import platform
import nest_asyncio
from pathlib import Path
from playwright.async_api import async_playwright

# Apply nest_asyncio to run in Jupyter
nest_asyncio.apply()

# Set up logging to capture and silence errors
logging.basicConfig(level=logging.INFO)
logging.getLogger('asyncio').setLevel(logging.CRITICAL)

# User data folder configuration
username = getpass.getuser()
system = platform.system()

if system == "Windows":
    chrome_path = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
    user_data_dir = f"C:\\Users\\{username}\\AppData\\Local\\Google\\Chrome\\User Data\\Default"
    profile_dir = f"C:\\Users\\{username}\\AppData\\Local\\Google\\Chrome\\User Data"
elif system == "Darwin":  # macOS
    chrome_path = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    user_data_dir = f"/Users/{username}/Library/Application Support/Google/Chrome/Default"
    profile_dir = f"/Users/{username}/Library/Application Support/Google/Chrome"
elif system == "Linux":  # Ubuntu/Debian
    chrome_path = "/usr/bin/google-chrome"
    user_data_dir = f"/home/{username}/.config/google-chrome/Default"
    profile_dir = f"/home/{username}/.config/google-chrome"
else:
    raise Exception("Unsupported operating system")

def print_instructions():
    print("To set up a new profile in Chromium, follow these steps:")
    print("1. Open Chromium and go to 'Settings'.")
    print("2. Scroll down to 'You and Google' and click 'Add person'.")
    print("3. Choose a name and an icon for the new profile.")
    print("4. Click 'Add' to create the new profile.")
    print("\nAfter you close the browser, you can find your new profile in the following directory based on your OS:")
    print(f"Windows: {profile_dir}")
    print(f"macOS: {profile_dir}")
    print(f"Linux: {profile_dir}")

async def open_browser():
    print_instructions()
    
    async with async_playwright() as plwt:
        context = await plwt.chromium.launch_persistent_context(
            user_data_dir,
            executable_path=chrome_path,
            headless=False,
            accept_downloads=True,
            ignore_default_args=["--enable-automation"],
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        )
        
        page = await context.new_page()
        await page.goto("https://example.com")
        
        try:
            # Keep the browser open until the user closes it, with a very long timeout
            await context.wait_for_event("close", timeout=24*60*60*1000)  # 24 hours
        except asyncio.TimeoutError:
            logging.error("Timeout while waiting for the browser to close.")
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
        finally:
            print(f"\nYour new browser profile can be found in the following directory:")
            print(f"{profile_dir}")

            # Provide clickable links to open the directory
            if system == "Windows":
                print(f"To open the directory, you can use this path in File Explorer: {profile_dir}")
            elif system == "Darwin":
                print(f"[Open Profile Directory](open_profile_directory_mac)")
            elif system == "Linux":
                print(f"[Open Profile Directory](open_profile_directory_linux)")

asyncio.run(open_browser())

def open_directory(path):
    if system == "Windows":
        import os
        os.startfile(path)
    elif system == "Darwin":
        import subprocess
        subprocess.Popen(["open", path])
    elif system == "Linux":
        import subprocess
        subprocess.Popen(["xdg-open", path])

# Automatically open the directory (uncomment the line below if you want this functionality)
# open_directory(profile_dir)